In [1]:
import cv2
import numpy as np
import vehicles
import time

In [2]:
cnt_up=0
cnt_down=0

In [3]:
cap=cv2.VideoCapture("cars1.m4v")

In [4]:
w=cap.get(3)
h=cap.get(4)
frameArea=h*w
areaTH=frameArea/400

print(frameArea,w,h)


230400.0 640.0 360.0


In [5]:
line_up=int(2*(h/5))
line_down=int(3*(h/5))

up_limit=int(1*(h/5))
down_limit=int(4*(h/5))

print("Red line y:",str(line_down))
print("Blue line y:",str(line_up))
line_down_color=(255,0,0)
line_up_color=(259,153,189)
pt1 =  [0, line_down]
pt2 =  [w, line_down]
pts_L1 = np.array([pt1,pt2], np.int32)
pts_L1 = pts_L1.reshape((-1,1,2))
pt3 =  [0, line_up]
pt4 =  [w, line_up]
pts_L2 = np.array([pt3,pt4], np.int32)
pts_L2 = pts_L2.reshape((-1,1,2))

pt5 =  [0, up_limit]
pt6 =  [w, up_limit]
pts_L3 = np.array([pt5,pt6], np.int32)
pts_L3 = pts_L3.reshape((-1,1,2))
pt7 =  [0, down_limit]
pt8 =  [w, down_limit]
pts_L4 = np.array([pt7,pt8], np.int32)
pts_L4 = pts_L4.reshape((-1,1,2))


Red line y: 216
Blue line y: 144


In [6]:
fgbg=cv2.createBackgroundSubtractorMOG2(detectShadows=True)

In [7]:
kernalOp = np.ones((3,3),np.uint8)
kernalOp2 = np.ones((5,5),np.uint8)
kernalCl = np.ones((11,11),np.uint8)


font = cv2.FONT_HERSHEY_SIMPLEX
cars = []
max_p_age = 5
pid = 1


In [8]:
while(cap.isOpened()):
    ret,frame=cap.read()
    for i in cars:
        i.age_one()
    fgmask=fgbg.apply(frame)
    fgmask2=fgbg.apply(frame)

    if ret==True:

        #Binarization
        ret,imBin=cv2.threshold(fgmask,200,255,cv2.THRESH_BINARY)
#         ret,imBin2=cv2.threshold(fgmask2,200,255,cv2.THRESH_BINARY)

        #OPening i.e First Erode the dilate
        mask=cv2.morphologyEx(imBin,cv2.MORPH_OPEN,kernalOp)
#         mask2=cv2.morphologyEx(imBin2,cv2.MORPH_CLOSE,kernalOp)

        #Closing i.e First Dilate then Erode
        mask=cv2.morphologyEx(mask,cv2.MORPH_CLOSE,kernalCl)
#         mask2=cv2.morphologyEx(mask2,cv2.MORPH_CLOSE,kernalCl)


        #Find Contours
        countours0,_=cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
        for cnt in countours0:
            area=cv2.contourArea(cnt)
            print(area)
            if area>areaTH:
                ####Tracking######
                m=cv2.moments(cnt)
                cx=int(m['m10']/m['m00'])
                cy=int(m['m01']/m['m00'])
                x,y,w,h=cv2.boundingRect(cnt)

                new=True
                if cy in range(up_limit,down_limit):
                    for i in cars:
                        if abs(x - i.getX()) <= w and abs(y - i.getY()) <= h:
                            new = False
                            i.updateCoords(cx, cy)

                            if i.going_UP(line_down,line_up)==True:
                                cnt_up+=1
                                print("ID:",i.getId(),'crossed going up at', time.strftime("%c"))
                            elif i.going_DOWN(line_down,line_up)==True:
                                cnt_down+=1
                                print("ID:", i.getId(), 'crossed going up at', time.strftime("%c"))
                            break
                        if i.getState()=='1':
                            if i.getDir()=='down'and i.getY()>down_limit:
                                i.setDone()
                            elif i.getDir()=='up'and i.getY()<up_limit:
                                i.setDone()
                        if i.timedOut():
                            index=cars.index(i)
                            cars.pop(index)
                            del i

                    if new==True: #If nothing is detected,create new
                        p=vehicles.Car(pid,cx,cy,max_p_age)
                        cars.append(p)
                        pid+=1

                cv2.circle(frame,(cx,cy),5,(0,0,255),-1)
                img=cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)

        for i in cars:
            cv2.putText(frame, str(i.getId()), (i.getX(), i.getY()), font, 0.3, i.getRGB(), 1, cv2.LINE_AA)




        str_up='UP: '+str(cnt_up)
        str_down='DOWN: '+str(cnt_down)
        frame=cv2.polylines(frame,[pts_L1],False,line_down_color,thickness=2)
        frame=cv2.polylines(frame,[pts_L2],False,line_up_color,thickness=2)
        frame=cv2.polylines(frame,[pts_L3],False,(255,255,255),thickness=1)
        frame=cv2.polylines(frame,[pts_L4],False,(255,255,255),thickness=1)
        cv2.putText(frame, str_up, (10, 40), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, str_up, (10, 40), font, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
        cv2.putText(frame, str_down, (10, 90), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, str_down, (10, 90), font, 0.5, (255, 0, 0), 1, cv2.LINE_AA)
        cv2.imshow('Frame',frame)
        cv2.imshow('Mask',mask)
#         cv2.imshow('Mask2',mask2)

        if cv2.waitKey(1)&0xff==ord('q'):
            break

    else:
        break

cap.release()
cv2.destroyAllWindows()



6032.0
9.0
7047.5
563.5
106.5
163.5
336.5
123.5
4.0
222.0
49.5
20.5
24.5
11.0
6.0
2.0
6.0
10.0
4.0
4.0
3324.0
4848.0
739.5
244.5
130.0
32.0
133.0
4.0
20.5
8.0
6.0
8.5
1520.5
4893.0
816.5
324.0
70.0
48.5
134.0
22.5
4.0
4.0
4.0
14.0
6.0
230.5
67.5
73.5
40.0
792.0
17.0
9.0
9.0
7244.0
1083.5
605.5
2.0
259.0
163.5
90.0
235.0
4.0
37.5
4.0
47.5
4.0
4.0
9.0
4.0
17.0
14.0
6.0
9.0
9.0
9.0
8408.5
633.0
2.0
297.0
188.5
130.0
182.0
4.0
4.0
120.0
10.5
15.0
8.0
24.0
8.0
22.0
6.0
6.0
9.0
7562.5
9.0
9.0
1407.0
778.0
4.0
156.0
355.5
123.5
244.5
33.0
14.5
4.0
4.0
37.0
4.0
12.0
16.0
14.0
30.0
19.5
54.0
6.0
16.0
11.0
151.5
1452.0
160.0
168.5
77.5
140.5
4.0
33.5
10.5
12.0
14.0
6.0
6.0
4.0
13.5
168.5
4.0
8.0
4.0
4.0
6404.5
1103.0
ID: 2 crossed going up at Wed May 26 20:20:04 2021
9.0
925.5
309.5
141.0
66.0
134.0
13.5
4.0
4.0
8.0
4.0
6.0
6555.5
4.0
4.0
1106.0
9.0
1005.0
431.5
234.0
208.5
204.0
6.0
4.0
25.0
18.5
4.0
31.0
9.0
4.0
5356.5
1553.5
9.0
1099.5
448.0
284.0
219.0
209.0
6.0
13.5
60.0
18.0
23.5
16.5
23.0

4672.5
6.0
6.0
9.0
421.0
42.5
25.0
318.0
4.0
52.5
4.0
52.0
11.0
6.0
20.0
8.5
4626.0
6.0
6.0
9.0
419.0
42.5
25.0
315.0
4.0
46.5
52.0
11.0
6.0
8.0
8.5
4927.0
315.5
396.0
50.5
21.0
315.5
4.0
43.5
16.0
45.5
14.5
6.0
12.0
8.5
4271.5
221.0
374.0
54.0
4.0
305.5
37.0
63.0
4.0
13.5
56.5
12.0
14.5
8.5
3554.5
223.5
473.0
7.5
79.5
14.5
10.5
293.0
76.5
4.0
47.0
16.5
6.0
10.5
8.5
3559.0
257.0
473.0
7.5
79.5
14.5
10.5
293.0
71.0
4.0
47.0
16.5
6.0
10.5
8.5
3468.0
209.0
475.5
6.5
79.5
14.5
10.5
293.0
71.0
4.0
47.0
16.5
6.0
10.5
8.5
3430.0
193.0
469.5
4.0
79.5
14.5
10.5
293.0
71.0
4.0
47.0
16.5
6.0
10.5
8.5
1578.5
374.5
402.5
38.0
59.0
29.0
8.0
234.5
12.0
71.5
4.0
46.0
21.5
6.0
8.5
522.5
453.5
676.5
102.5
67.5
8.0
32.0
185.0
83.0
4.0
29.5
51.5
260.5
520.5
133.5
71.5
45.0
158.0
6.0
73.5
39.0
30.0
260.5
520.5
131.0
62.5
6.0
45.0
141.0
6.0
74.0
39.0
29.5
260.5
520.5
131.0
71.5
45.0
141.0
6.0
74.0
38.5
29.5
260.5
540.0
131.0
71.5
45.0
141.0
6.0
74.0
38.5
29.5
178.0
656.0
168.0
72.5
4.0
8.5
6.0
150.0
104.0
5

8.0
936.5
ID: 25 crossed going up at Wed May 26 20:20:07 2021
207.5
61.0
229.0
229.5
18.0
42.0
4.0
335.0
638.5
22.5
67.5
11.0
4.0
6.0
52.0
6.0
28.0
12.0
47.0
8.5
9.0
136.5
8.0
936.5
207.5
61.0
229.0
229.5
18.0
42.0
4.0
335.0
615.0
22.5
67.5
11.0
4.0
6.0
52.0
6.0
28.0
12.0
47.0
8.5
9.0
136.5
8.0
936.5
207.5
61.0
229.0
212.5
18.0
42.0
4.0
332.0
615.0
22.5
67.5
11.0
4.0
6.0
50.5
6.0
28.0
12.0
47.0
8.5
9.0
136.5
8.0
1206.0
362.5
14.0
303.0
206.0
11.0
8.0
4.0
170.5
582.0
35.5
8.5
46.5
4.0
14.0
4.0
4.0
30.5
34.0
12.0
47.0
8.5
9.0
108.0
8.0
1452.0
625.0
174.5
211.5
4.0
8.0
91.0
163.0
541.5
37.5
37.0
4.0
4.0
6.0
45.0
4.0
32.5
12.0
47.0
8.5
9.0
108.0
8.0
4.0
1623.0
921.5
6.0
493.0
299.0
41.5
16.5
271.0
609.0
25.0
6.0
30.5
6.0
6.0
6.0
46.0
4.0
34.0
12.0
47.0
8.5
9.0
108.0
8.0
4.0
1623.0
913.0
6.0
493.0
313.0
41.5
14.0
271.0
612.0
25.0
4.0
31.0
4.0
6.0
6.0
43.0
4.0
34.0
12.0
47.0
8.5
9.0
108.0
8.0
4.0
1623.0
920.5
6.0
492.0
313.0
41.5
271.0
612.0
17.0
4.0
31.0
4.0
6.0
6.0
43.0
4.0
34.0
12.0
47.0


2102.5
6.0
10.0
2230.5
496.5
8.0
90.0
87.5
6.0
8.0
4.0
4.0
290.5
4.0
10.0
31.0
4.0
14.5
8.0
87.0
4.0
15.0
4.0
4.0
2102.5
6.0
10.0
2230.5
496.5
8.0
90.0
87.5
6.0
8.0
4.0
290.0
4.0
10.0
31.0
4.0
14.5
8.0
82.0
4.0
15.0
4.0
4.0
2102.5
10.0
2230.5
496.5
8.0
86.0
87.5
6.0
8.0
4.0
284.5
8.0
31.0
4.0
14.5
8.0
82.0
4.0
15.0
4.0
4.0
153.5
4.0
10.0
2843.0
591.5
6.0
61.5
130.5
159.0
6.0
199.0
4.0
60.5
23.5
4.0
10.0
84.0
4.0
15.0
4.0
4.0
10.0
4.0
3131.0
635.0
4.0
78.0
258.0
204.0
71.0
8.0
25.0
14.0
4.0
57.0
4.0
22.5
10.0
87.0
4.0
15.0
4.0
4.0
17.0
3665.5
640.0
4.0
94.0
215.0
64.0
6.0
8.0
184.5
4.0
10.0
6.0
27.0
6.0
20.5
4.0
8.0
93.5
4.0
15.0
4.0
4.0
10.0
3719.0
640.0
4.0
94.0
215.0
64.0
6.0
8.0
184.5
4.0
8.0
6.0
27.0
6.0
20.5
4.0
8.0
93.5
4.0
15.0
4.0
4.0
10.0
3691.5
640.0
4.0
84.0
214.0
64.0
6.0
6.0
184.5
8.0
6.0
27.0
6.0
20.5
4.0
8.0
93.5
13.5
3691.5
640.0
4.0
70.5
214.0
64.0
6.0
6.0
184.5
8.0
6.0
27.0
20.5
4.0
8.0
93.5
13.5
4254.5
13.0
724.5
321.0
96.5
4.0
351.0
91.0
8.0
25.5
4.0
41.5
12.0
91.0


3105.5
4493.5
13.0
657.0
1317.5
547.5
178.5
57.5
6.0
198.5
102.0
62.0
4.0
4.0
3105.5
4491.5
13.0
657.0
1311.0
547.5
178.5
57.5
6.0
198.5
102.0
63.5
4.0
4.0
3105.5
4491.5
8.5
657.0
1311.0
547.5
178.5
57.5
198.5
99.5
63.5
4.0
4.0
3105.5
4466.5
6.0
653.5
1311.0
547.5
178.5
57.5
198.5
99.5
63.5
4.0
4.0
531.0
3424.5
4.0
924.5
52.5
1490.5
594.0
197.0
90.0
113.0
214.5
56.0
2457.5
825.0
6.0
1665.0
615.0
170.5
62.5
100.5
226.0
56.0
9.0
4.0
1689.5
777.0
1773.5
ID: 53 crossed going up at Wed May 26 20:20:11 2021
610.0
198.5
65.0
12.0
42.5
199.5
151.0
4.0
1689.5
4.0
772.0
1773.5
609.0
198.5
32.5
6.0
42.5
199.5
151.0
4.0
1689.5
4.0
772.0
1773.5
609.0
198.5
32.5
6.0
42.5
196.5
151.0
4.0
1689.5
4.0
759.5
1773.5
609.0
198.5
26.5
6.0
42.5
196.5
151.0
15.5
157.5
4.0
685.0
2149.5
459.0
242.5
4.0
14.0
4.0
240.5
143.0
126.5
15.5
1780.0
2338.5
500.0
253.0
6.0
14.0
35.5
210.0
4.0
141.0
2075.5
2480.5
736.0
206.0
35.0
305.5
114.5
73.5
4.0
2073.5
2480.5
736.0
206.0
35.0
4.0
305.5
19.5
73.5
4.0
2073.5
2480.5
726

...

8.5
9.0
5218.0
2606.0
108.5
308.5
174.5
24.0
57.0
8.0
4.0
6.0
89.0
20.0
93.5
6.0
22.5
6.0
6.0
75.5
8.5
9.0
5217.0
2613.5
108.5
308.5
174.5
20.0
53.0
8.0
4.0
6.0
78.0
20.0
93.5
6.0
22.5
6.0
6.0
75.0
8.5
9.0
5214.5
2613.5
108.5
308.5
174.5
20.0
53.0
8.0
4.0
4.0
6.0
13.0
20.0
80.0
6.0
22.5
6.0
6.0
75.0
4579.5
23.5
2451.0
108.0
453.0
176.0
14.0
50.5
10.5
4.0
26.0
20.0
90.0
4.0
22.0
4.0
75.0
2380.0
3414.5
104.0
192.0
26.0
529.0
45.0
10.0
4.0
161.5
4.0
6.0
23.0
6.0
80.5
1641.0
3954.5
115.5
514.0
22.0
249.5
60.5
10.5
23.0
8.0
87.0
23.5
6.0
85.5
1641.0
3950.5
115.5
514.0
22.0
249.5
60.5
10.5
23.0
8.0
87.0
23.5
6.0
85.5
1641.0
3950.5
115.5
514.0
22.0
249.5
55.0
10.5
4.0
12.5
8.0
87.0
23.5
6.0
85.5
1640.0
3946.5
115.5
514.0
22.0
249.5
55.0
10.5
4.0
12.5
8.0
87.0
23.5
6.0
85.5
457.5
4.0
3204.0
141.5
547.5
10.0
229.0
66.0
6.0
4.0
36.0
22.5
95.0
23.5
10.0
8.0
90.5
22.5
1888.0
114.5
541.0
24.5
310.5
35.0
59.5
4.0
4.0
8.0
6.0
12.5
65.0
21.0
10.0
4.0
74.5
1246.5
133.0
601.0
61.5
352.0
51.5
45.5
4.0
6.

1864.5
626.5
1017.5
639.5
84.0
6.0
334.0
34.0
4.0
16.5
6.0
4.0
6.0
18.0
16.0
62.0
1864.5
626.5
1017.5
639.5
84.0
6.0
334.0
34.0
4.0
16.5
6.0
4.0
6.0
15.0
16.0
62.0
14.5
1167.5
860.5
1226.0
725.5
86.0
299.0
4.0
6.0
8.0
42.5
14.0
13.5
4.0
12.0
21.0
63.0
1112.5
644.5
1296.5
776.5
149.0
79.0
4.0
19.5
130.5
4.0
4.0
13.5
4.0
12.0
19.0
65.0
1112.5
644.5
1296.5
776.5
149.0
79.0
4.0
19.5
130.5
4.0
4.0
13.5
4.0
12.0
19.0
65.0
1104.5
644.5
1296.5
776.5
149.0
71.0
4.0
19.5
130.5
4.0
4.0
11.5
4.0
12.0
19.0
65.0
379.5
232.0
1545.5
866.5
160.0
86.5
6.0
8.0
169.5
35.0
4.0
11.5
4.0
4.0
14.5
20.5
75.5
379.5
232.0
1544.0
866.5
148.0
86.5
6.0
4.0
204.0
35.0
4.0
9.0
4.0
4.0
14.5
20.5
75.5
1789.0
998.5
227.0
177.0
6.0
4.0
34.0
6.0
128.0
10.5
4.0
4.0
12.0
8.0
24.0
73.0
1936.0
1057.0
ID: 133 crossed going up at Wed May 26 20:20:25 2021
234.0
127.5
86.0
43.5
29.0
8.0
10.5
4.0
10.0
14.5
6.0
28.5
4.0
65.5
1936.0
1057.0
234.0
127.5
86.0
45.5
29.0
4.0
10.5
4.0
4.0
14.5
4.0
28.5
65.5
1925.0
1057.0
239.0
127.5
86.0


240.0
279.0
24.0
239.5
174.5
16.5
70.5
8.0
100.0
1352.0
4052.5
239.0
276.0
24.0
239.5
173.0
16.5
70.5
8.0
100.0
533.5
3204.0
260.5
22.0
837.5
177.0
31.0
72.5
4.0
4.0
127.5
533.5
3204.0
260.5
22.0
825.5
177.0
31.0
72.5
4.0
4.0
127.5
533.5
3204.0
260.5
22.0
825.5
177.0
14.0
72.5
4.0
4.0
126.0
2128.0
217.0
322.0
6.0
225.0
261.0
12.5
67.0
4.0
4.0
123.5
242.0
301.5
32.0
960.0
318.5
4.0
71.5
8.0
142.0
242.0
301.5
32.0
960.0
330.5
4.0
60.5
8.0
142.0
335.0
39.0
994.0
450.0
53.5
8.0
109.5
335.0
39.0
991.5
450.0
54.5
8.0
112.5
335.0
39.0
978.5
414.0
54.5
8.0
107.0
341.5
32.0
968.5
304.5
57.0
4.0
87.5
458.0
44.0
1062.0
234.0
299.0
34.5
4.0
49.5
19.0
114.0
458.0
44.0
1062.0
165.5
299.0
34.5
4.0
49.5
19.0
114.0
441.0
45.0
1123.0
141.0
247.0
18.5
4.0
4.0
6.0
66.5
8.0
4.0
110.0
441.0
43.0
1123.0
141.0
247.0
18.5
4.0
4.0
6.0
66.5
8.0
4.0
95.5
441.0
43.0
1122.0
139.5
247.0
18.5
4.0
4.0
48.5
6.0
8.0
4.0
81.5
456.0
49.0
1186.0
178.5
463.0
4.0
6.0
50.5
9.0
8.0
4.0
95.0
549.5
57.0
1027.5
ID: 146 crossed go

3005.5
444.5
624.0
107.0
92.5
17.5
190.0
62.0
42.0
16.0
8.0
1675.5
1599.5
3231.5
461.5
811.0
48.0
2.0
115.0
145.5
66.5
47.5
16.0
8.0
1677.5
1599.5
3231.5
461.5
812.0
48.0
96.5
2.0
14.5
145.5
65.5
47.5
16.0
8.0
1677.5
1599.5
3227.0
461.5
812.0
48.0
96.5
2.0
4.0
145.5
65.5
47.5
14.0
8.0
1677.5
1599.5
3227.0
461.5
806.0
48.0
96.5
2.0
4.0
145.5
65.5
47.5
14.0
8.0
600.0
850.0
3506.5
491.5
1132.0
30.0
123.5
12.0
218.0
63.0
38.0
14.0
8.0
4161.0
632.0
1043.5
153.5
96.5
6.0
157.0
58.5
42.0
14.0
12.0
4532.0
672.5
1072.0
163.0
6.0
131.5
222.5
61.5
4.0
40.5
14.0
10.0
4537.0
671.5
1072.0
163.0
6.0
131.5
222.5
61.5
4.0
40.5
14.0
10.0
4535.5
671.5
1072.0
163.0
6.0
131.5
222.5
61.5
4.0
40.5
14.0
10.0
4535.5
671.5
1061.5
163.0
6.0
131.5
222.5
61.5
4.0
40.5
14.0
10.0
4937.5
712.0
1340.0
175.5
204.0
4.0
178.5
54.5
38.5
12.0
12.0
6154.0
824.0
1137.0
276.0
215.0
130.0
6.0
67.0
8.0
38.0
8.0
12.0
6156.5
481.0
873.5
531.0
178.5
11.0
204.5
190.0
55.5
40.5
16.5
6.0
12.0
6162.0
480.0
874.5
531.0
178.5
11.0
204.5

2064.0
52.5
398.5
600.0
91.5
82.5
6.0
191.5
23.0
139.0
4.0
35.0
18.0
2417.0
48.0
310.5
539.5
86.0
107.5
31.5
66.5
19.0
26.5
142.5
36.0
2417.0
293.5
48.0
539.5
86.0
110.5
31.5
66.5
19.0
26.5
139.0
36.0
2347.5
293.5
48.0
554.5
86.0
110.5
31.5
66.5
19.0
26.5
137.0
36.0
2347.5
293.5
48.0
554.5
86.0
108.5
8.5
66.5
19.0
26.5
137.0
36.0
2706.0
62.0
425.0
543.0
70.5
84.0
8.0
4.0
69.5
19.0
12.0
149.0
4.0
4.0
41.0
3687.0
64.0
459.0
704.0
4.0
59.5
73.0
121.0
24.0
14.0
77.0
4.0
4.0
39.0
4138.0
112.0
423.5
707.0
60.0
79.0
25.0
58.5
25.0
16.5
147.5
6.0
4.0
35.0
4138.0
112.0
432.5
718.0
60.0
69.0
27.5
60.0
25.0
16.5
145.5
6.0
4.0
44.5
4138.0
112.0
429.5
718.0
60.0
69.0
23.0
57.5
25.0
16.5
145.5
6.0
4.0
44.5
4075.5
112.0
429.5
690.0
60.0
69.0
23.0
57.5
25.0
16.5
145.5
6.0
44.5
4262.0
110.5
397.5
749.5
66.0
62.0
23.5
57.5
10.0
144.5
19.5
38.0
3152.5
78.5
581.5
837.5
4.0
72.0
73.0
27.0
14.0
150.5
74.5
4.0
42.5
2590.5
132.5
606.0
897.5
155.5
88.0
33.0
57.0
155.5
10.5
4.0
4.0
46.0
2587.5
132.5
606.0
874.0